In [ ]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
!./sharingan_export.sh

In [ ]:
fn = "../data/input/evaluation/001000.bin"

raw_data=np.fromfile(fn, np.float32)
print("raw data shape=", raw_data.shape)

raw_data_reshaped = np.reshape(raw_data,[2,1,-1,1])

input  = raw_data_reshaped[0].reshape([1,1,-1,1])
target = raw_data_reshaped[1].reshape([1,1,-1,1])

In [ ]:
interpreter = tf.contrib.lite.Interpreter(model_path="../data/output/frozen_model/model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
print(input_details)
print(output_details)

interpreter.set_tensor(input_details[0]['index'], input)
interpreter.invoke()
interpreter_output = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
tf.reset_default_graph()

with tf.gfile.GFile("../data/output/frozen_model/frozen_model.pb", "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Graph().as_default() as gf:
    tf.import_graph_def(graph_def, name="prefix")
    graph = gf

#for op in graph.get_operations():
#    print(op.name)

X=graph.get_tensor_by_name('prefix/input:0')
output_node = graph.get_tensor_by_name('prefix/generator/Tanh:0')
with tf.Session(graph=graph) as sess:
    frozen_output=output_node.eval({X: input})

In [ ]:
x=np.arange(1024)
plt.plot(x, input[0,0,:,0])
plt.plot(x, target[0,0,:,0])
plt.plot(x, interpreter_output[0,0,:,0], '.')
plt.plot(x, frozen_output[0,0,:,0], '-')

In [ ]:
tf.__version__